In [2]:
import cv2 as cv2
import cv2.aruco as aruco
import sys
from matplotlib import pyplot as plt
import numpy as np
from sympy import symbols, Eq, solve
import glob
import os

In [3]:
#Example co-ordinates were being used as object points for the PnP solver
obj_points = np.array([[0.0, 0.0, 0.0], [0.0, 2.0, 0.0],[2.0, 2.0, 0.0],[2.0, 0.0, 0.0]])
#get aruco tags 
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_1000)
#Define marker length and marker seperation (these parameters doesn't effect the answer)
marker_length = 2.0
marker_seperation = 0.6
#Define arucoParams
arucoParams = aruco.DetectorParameters_create()
#read the image
img= cv2.imread('/Users/pavan/Downloads/OpenCV/single_tag4.jpg')
if img is not None:
    img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #Convert image to gray scale
    #store corners
    corners, ids, rejected = aruco.detectMarkers(img_gray, aruco_dict, parameters=arucoParams)
else:
    pass
#extract corners
corners = corners[0]
#Define k_matrix
k_matrix = np.array([[3.25422161e+03, 0.00000000e+00, 2.03416424e+03],
 [0.00000000e+00, 3.25189597e+03, 1.49339521e+03],
 [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
#define distortion_matrix
dist_matrix = np.array([[ 2.19861352e-01, -1.38637645e+00,  2.01013731e-04,  3.10245273e-05,
   2.85684735e+00]])
#PnP solver returns 3*1 rotation vector
ret, rvecs, tvecs = cv2.solvePnP(obj_points, corners, k_matrix, dist_matrix)
#Use Rodrigues function to find rotation matrix
rotation_mat, jacobian = cv2.Rodrigues(rvecs)
#Concatenate Rotation and translation matrices to get extrinsic_matrix
extrinsic_matrix = cv2.hconcat((rotation_mat, tvecs))
#Calculate homography
homography_matrix = np.matmul(k_matrix,extrinsic_matrix)

#Remaining 4 points in world frame homogeneous coordinates
cube_top1 = np.array([[0],[0],[2],[1]])
cube_top2 = np.array([[2],[0],[2],[1]])
cube_top3 = np.array([[0],[2],[2],[1]])
cube_top4 = np.array([[2],[2],[2],[1]])
#Calculating homogeneous image coordinates
image_hom1 = np.matmul(homography_matrix,cube_top1)
image_hom2 = np.matmul(homography_matrix,cube_top2)
image_hom3 = np.matmul(homography_matrix,cube_top3)
image_hom4 = np.matmul(homography_matrix,cube_top4)
dv1 = image_hom1[2]
dv2 = image_hom2[2]
dv3 = image_hom3[2]
dv4 = image_hom4[2]
#Performing perspetive division and converting into tuples for cv2.line
image_top1 = np.divide(image_hom1,dv1)
image_top2 = np.divide(image_hom2,dv2)
image_top3 = np.divide(image_hom3,dv3)
image_top4 = np.divide(image_hom4,dv4)
image_top1_x = image_top1[0]
image_top1_y = image_top1[1]
y1 = (image_top1_x,image_top1_y)
top11 = round(y1[0][0])
top12 = round(y1[1][0])
top1 = (top11,top12)
print(top1)
image_top2_x = image_top2[0]
image_top2_y = image_top2[1]
y2 = (image_top2_x,image_top2_y)
top21 = round(y2[0][0])
top22 = round(y2[1][0])
top2 = (top21,top22)
print(top2)
image_top3_x = image_top3[0]
image_top3_y = image_top3[1]
y3 = (image_top3_x,image_top3_y)
top31 = round(y3[0][0])
top32 = round(y3[1][0])
top3 = (top31,top32)
print(top3)


image_top4_x = image_top4[0]
image_top4_y = image_top4[1]
y4 = (image_top4_x,image_top4_y)
top41 = round(y4[0][0])
top42 = round(y4[1][0])
top4 = (top41,top42)
print(top4)
#Draw lines to join corners of cube
cv2.line(img, top1, top2, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, top1, top3, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, top2, top4, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, top3, top4, (0, 255, 0), thickness=3, lineType=8)
corners_2 = np.zeros((4,2))
np.copyto(corners_2,corners[0])
#Extract tuples from the corners detected by aruco tags as well
tup1 =(int(corners_2[0][0]),int(corners_2[0][1]))
print(tup1)
tup3 =(int(corners_2[1][0]),int(corners_2[1][1]))
print(tup3)
tup4 =(int(corners_2[2][0]),int(corners_2[2][1]))
print(tup4)
tup2 =(int(corners_2[3][0]),int(corners_2[3][1]))
print(tup2)
#Complete the cube
cv2.line(img, tup1, tup2, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, tup1, tup3, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, tup2, tup4, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, tup3, tup4, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, top1, tup1, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, top2, tup2, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, top3, tup3, (0, 255, 0), thickness=3, lineType=8)
cv2.line(img, top4, tup4, (0, 255, 0), thickness=3, lineType=8)
#Show image
cv2.imshow("test", img)
#Wait and Destroy Image
cv2.waitKey(1000)
cv2.destroyAllWindows()

(2831, 1086)
(2663, 1431)
(3067, 1375)
(2862, 1728)
(3198, 1097)
(3459, 1375)
(3231, 1713)
(3009, 1428)
